In [ ]:
#!pip uninstall -y torch torchvision torchaudio accelerate transformers sentence_transformers
#!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 accelerate==0.25.0 transformers==4.36.0 sentence_transformers==2.7.0
#!pip install eland elasticsearch

In [ ]:
#!pip install sentencepiece

In [ ]:
import sentencepiece
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient

In [ ]:
# 1. ES 클라이언트 정의
import os
from dotenv import load_dotenv
from elasticsearch import Elasticsearch

load_dotenv()

url = os.getenv("ELASTIC_CLOUD_URL")
api_id = os.getenv("ELASTIC_API_ID")
api_key = os.getenv("ELASTIC_API_KEY")

client = Elasticsearch(
    url,
    api_key=(api_id, api_key)
)

print(client.info())

In [ ]:
# 2. 임베딩 모델 다운로드
hf_model_id='intfloat/multilingual-e5-base'
tm = TransformerModel(model_id=hf_model_id, task_type="text_embedding")

es_model_id = tm.elasticsearch_model_id()

tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

In [ ]:
# 4. ES ML 노드에 Embedding 모델 deploy

ptm = PyTorchModel(client, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config)

m = MlClient.get_trained_models(client, model_id=es_model_id)

MlClient.start_trained_model_deployment(client, model_id=es_model_id)

In [ ]:
# 5. ES cluster에 배포된 Embedding 모델 확인
stats = MlClient.get_trained_models_stats(client, model_id=es_model_id)
stats.body['trained_model_stats'][0]['model_id']

In [ ]:
# 6. 샘플 embedding
docs = [
    {
        "text_field": "안녕하세요. 사과 입니다."
    }
 ]

result = MlClient.infer_trained_model(client, model_id=es_model_id, docs=docs)

doc_0_vector = result['inference_results'][0]['predicted_value']
print(len(doc_0_vector))

doc_0_vector